In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Group Theory with Z3

In this notebook we show how to investigate topics in [group theory](https://en.wikipedia.org/wiki/Group_theory) using the constraint solver `Z3`.  In particular, 
we want to show that not every group is *commutative* by constructing a group in which the commutative law does not hold.

In [2]:
!pip install z3-solver

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import z3

A [group](https://en.wikipedia.org/wiki/Group_(mathematics)) is defined as a triple 
$$ \mathcal{G} = \langle G, \mathrm{e}, \circ \rangle $$
such that
- $G$ is a non-empty set,
- $\mathrm{e}$ is an element from $G$, and
- $\circ:G \times G \rightarrow G$ is a binary operation on $G$ such that the following axioms are satisfied:
  * $\forall x: \mathrm{e} \circ  x = x$,
  * $\forall x: \exists{y}: y \circ x = \mathrm{e}$,
  * $\forall x: \forall y: \forall z: (x \circ y) \circ z = x \circ (y \circ z)$.
  
The group $\mathcal{G}$ is *commutative* if, additionally, the following holds:
  $$\forall x: \forall y: x \circ y = y \circ x. $$
For example, the triple $\langle \mathbb{Z}, 0, + \rangle$ is a commutative group. 
  
It is natural to ask whether every group is commutative, i.e. whether the formula 
$\forall x: \forall y: x \circ y = y \circ x$ is a logical consequence of the three group axioms.
To this end we can try to prove the following
$$ \{ \forall x: \mathrm{e} \circ  x = x,\; \forall x: \exists{y}: y \circ x = \mathrm{e},\;
      \forall x: \forall y: \forall z: (x \circ y) \circ z = x \circ (y \circ z) \} \models \forall x: \forall y: x \circ y = y \circ x
$$
using automatic theorem proving.  

In [4]:
!cat ../../Vampire/non-commutative-group.tptp

% We try to prove that every group is commutative.

% e is the neutral element.
fof(a1, axiom, ! [X] : mult(e,X) = X).

% For every element X there is a left inverse element Y.
fof(a2, axiom, ! [X] : ? [Y]: mult(Y, X) = e).

% The law of associativity holds.
fof(a3, axiom, ! [X,Y,Z] : mult(mult(X, Y), Z) = mult(X, mult(Y, Z))).

% Question: Does the law of commutativity hold?
fof(commute, conjecture, ! [X, Y] : mult(X, Y) = mult(Y, X)).


In [5]:
!vampire --input_syntax tptp ../../Vampire/non-commutative-group.tptp

% Time limit reached!
% ------------------------------
% Version: Vampire 4.7 (commit )
% Termination reason: Time limit
% Termination phase: Saturation

% Memory used [KB]: 4236775
% Time elapsed: 60.100 s
% ------------------------------
% ------------------------------


However, as this attempt produces no result we can instead try to refute the hypothesis that every group is commutative by constructing a group that is not commutative.  We will try to find a *finite* non-commutative group.

In a finite group with $n$ elements we can assume without loss of generality that the elements
are the natural numbers $\{ 0, 1, 2, \cdots, n-1 \}$.  Furthermore, by reordering the elements 
we can assume that $0$ is the neutral element $\mathrm{e}$, i.e. we have $\mathrm{e} = 0$.

Therefore, in order to construct a group we only have to find a binary operation $\circ$ on the set
$\{ 0, \cdots, n-1 \}$ that satisfies the three axioms specifying a group.  We will specify this operation 
as a dictionary `Multiply` that maps pairs of the form $\langle x, y \rangle \in \{ 0, \cdots, n-1 \} \times \{ 0, \cdots, n-1 \}$
into the set $\{ 0, \cdots, n-1 \}$.  The dictionary `Multiply` can be represented by the following set of `Z3` variables:

In [8]:
n = 3
Variables = { (row, col): z3.Int(f'V{row}{col}') for col in range(n) 
                                                 for row in range(n) 
            }
Variables

{(0, 0): V00,
 (1, 0): V10,
 (2, 0): V20,
 (0, 1): V01,
 (1, 1): V11,
 (2, 1): V21,
 (0, 2): V02,
 (1, 2): V12,
 (2, 2): V22}

The idea is that we have
$$ x \circ y = z \;\Longleftrightarrow\; \texttt{Variables}[x, y] = z.$$

The axiom 
$$ \forall x: \mathrm{e} \circ x = x $$
that $0$ is the left neutral element of the binary operation $\circ$ translates into the formula:
$$ \texttt{Multiply}[0, x] = x \quad \mbox{for all $x \in \{0, \cdots, n-1\}$.} $$
The function `left_neutral` takes two arguments:
* `n` is the size of the group.
* `Variables` is the dictionary that encodes the binary operation of the group.

The function returns a set of constraints specifying that $0$ is the *left neutral element* of the group.

In [6]:
def left_neutral(n, Variables):
    return { Variables[0, x] == x for x in range(n) }

In [9]:
left_neutral(3, Variables)

{V00 == 0, V01 == 1, V02 == 2}

The axiom
$$ \forall x: \exists y: y \circ x = \mathrm{e} $$
that every element $x$ has a left inverse element $y$ translates into the formula
$$ \forall x \in \{0, \cdots, n-1\}: \exists y \in \{0, \cdots, n-1\}: \texttt{Multiply}[y, x] = 0. $$

The function `left_inverse` takes two arguments:
* `n` is the size of the group.
* `Variables` is the dictionary that encodes the binary operation of the group.

The function returns a set of constraints that specify that every element has a left inverse element.
Since the neutral element $0$ is already its own left inverse, for reasons of efficiency this element is excluded.
Furthermore, we already know that the neutral element $0$ can never be the left inverse element of an
element that is different from $0$.

In [10]:
def left_inverse(n, Variables):
    Constraints = set()
    for x in range(1, n):
        Constraints |= { z3.Or({ Variables[y, x] == 0 for y in range(1, n) }) }
    return Constraints

In [11]:
left_inverse(3, Variables)

{Or(V12 == 0, V22 == 0), Or(V21 == 0, V11 == 0)}

The axiom
$$ \forall x: \forall y: \forall z: (x \circ y) \circ z = x \circ (y \circ z) $$
translates into the formula
$$ \forall x \in G: 
   \forall y \in G:
   \forall z \in G:
   \forall xy \in G:
   \forall zy \in G:
     \bigl(\texttt{Multiply}[x, y] = u \wedge \texttt{Multiply}[y, z] = v \rightarrow \texttt{Multiply}[u, z] = \texttt{Multiply}[x, v]\bigr), $$
where we have defined $G = \{0, \cdots n-1\}$.

The function `associative` takes two arguments:
* `n` is the size of the group.
* `Variables` is the dictionary that encodes the binary operation of the group.

The function returns a set of constraints that specify that the binary operation is associative.

In [13]:
def associative(n, Variables):
    L = range(0, n)
    Constraints = set()
    for x in L:
        for y in L:
            for z in L:
                for xy in L:
                    for yz in L:
                        Constraints |= { z3.Implies(z3.And(Variables[x, y] == xy, Variables[y, z] == yz),
                                                    Variables[xy, z] == Variables[x, yz]) 
                                       }
    return Constraints

In [14]:
associative(3, Variables)

{Implies(And(V00 == 0, V00 == 0), V00 == V00),
 Implies(And(V00 == 0, V00 == 1), V00 == V01),
 Implies(And(V00 == 0, V00 == 2), V00 == V02),
 Implies(And(V00 == 0, V01 == 0), V01 == V00),
 Implies(And(V00 == 0, V01 == 1), V01 == V01),
 Implies(And(V00 == 0, V01 == 2), V01 == V02),
 Implies(And(V00 == 0, V02 == 0), V02 == V00),
 Implies(And(V00 == 0, V02 == 1), V02 == V01),
 Implies(And(V00 == 0, V02 == 2), V02 == V02),
 Implies(And(V00 == 1, V00 == 0), V10 == V00),
 Implies(And(V00 == 1, V00 == 1), V10 == V01),
 Implies(And(V00 == 1, V00 == 2), V10 == V02),
 Implies(And(V00 == 1, V01 == 0), V11 == V00),
 Implies(And(V00 == 1, V01 == 1), V11 == V01),
 Implies(And(V00 == 1, V01 == 2), V11 == V02),
 Implies(And(V00 == 1, V02 == 0), V12 == V00),
 Implies(And(V00 == 1, V02 == 1), V12 == V01),
 Implies(And(V00 == 1, V02 == 2), V12 == V02),
 Implies(And(V00 == 2, V00 == 0), V20 == V00),
 Implies(And(V00 == 2, V00 == 1), V20 == V01),
 Implies(And(V00 == 2, V00 == 2), V20 == V02),
 Implies(And(

The function `non_commute` takes two arguments:
* `n` is the size of the group.
* `Variables` is the dictionary that encodes the binary operation of the group.

The function returns a set of constraints that specify that the binary operation $\circ$ is 
not commutative, i.e. there is some $x$ and some $y$ such that $x \circ y \not= y \circ x$.

In [15]:
def non_commute(n, Variables):
    Formulas = set()
    for x in range(1, n):
        for y in range(1, n):
            if x != y:
                Formulas |= { Variables[x, y] != Variables[y, x] }
    return { z3.Or(Formulas) }

In [16]:
non_commute(3, Variables)

{Or(V12 != V21, V21 != V12)}

All group elements are from the set $\{0,1,\cdots,n-1\}$.

The function `in_range` takes two arguments:
* `n` is the size of the group.
* `Variables` is the dictionary that encodes the binary operation of the group.

The function returns a set of constraints that specify that the values of all variables are elements of
the set $\{0,1,\cdots,n-1\}$.

In [17]:
def in_range(n, Variables):
    Constraints = set()
    for x in range(n):
        for y in range(n):
            Constraints |= { 0 <= Variables[x, y], Variables[x, y] < n }
    return Constraints

The function `all_constraints` returns a CSP that encodes the given sudoku as a CSP.

In [18]:
def all_constraints(n, Variables):
    Constraints  = left_neutral(n, Variables)
    Constraints |= left_inverse(n, Variables)
    Constraints |= associative (n, Variables)
    Constraints |= non_commute (n, Variables)
    Constraints |= in_range    (n, Variables)
    return Constraints

In [19]:
len(all_constraints(3, Variables))

267

The function `solve` receives one argument:
- `n` is the size of the group that should be constructed.

The function tries to construct a non-commutative group of size `n`.  It returns a solution if such a group exists.
Otherwise, `None` is returned.

In [20]:
def solve(n):
    Variables = { (row, col): z3.Int(f'V{row}{col}') for col in range(n) 
                                                     for row in range(n) }
    S = z3.Solver()
    Constraints = all_constraints(n, Variables)
    print(f'Number of constraints: {len(Constraints)}.')
    S.add(Constraints)
    result = str(S.check())
    if result == 'sat':
        M = S.model()
        Solution = { f'V{row}{col}': M[Variables[row, col]] for row in range(n)
                                                            for col in range(n)
                   }
        return Solution

The function `find_non_commutative_group` tries to find a non-commutative group.

In [21]:
def find_non_commutative_group():
    n = 1
    while True:
        print(f'Testing n = {n} ...')
        Solution = solve(n)
        if Solution:
            return Solution
        n += 1

In [22]:
%%time
Solution = find_non_commutative_group()
Solution

Testing n = 1 ...
Number of constraints: 5.
Testing n = 2 ...
Number of constraints: 44.
Testing n = 3 ...
Number of constraints: 267.
Testing n = 4 ...
Number of constraints: 1064.
Testing n = 5 ...
Number of constraints: 3185.
Testing n = 6 ...
Number of constraints: 7860.
CPU times: user 1.29 s, sys: 36.1 ms, total: 1.33 s
Wall time: 1.35 s


{'V00': 0,
 'V01': 1,
 'V02': 2,
 'V03': 3,
 'V04': 4,
 'V05': 5,
 'V10': 1,
 'V11': 2,
 'V12': 0,
 'V13': 4,
 'V14': 5,
 'V15': 3,
 'V20': 2,
 'V21': 0,
 'V22': 1,
 'V23': 5,
 'V24': 3,
 'V25': 4,
 'V30': 3,
 'V31': 5,
 'V32': 4,
 'V33': 0,
 'V34': 2,
 'V35': 1,
 'V40': 4,
 'V41': 3,
 'V42': 5,
 'V43': 1,
 'V44': 0,
 'V45': 2,
 'V50': 5,
 'V51': 4,
 'V52': 3,
 'V53': 2,
 'V54': 1,
 'V55': 0}

## Graphical Representation

In [23]:
import math

In [24]:
def show(Solution):
    n = round(math.sqrt(len(Solution)))
    # Determine the width of the widest element in the matrix
    # Create the top and bottom of the matrix
    top_line = '╔'
    for i in range(n - 1):
        top_line += '═' * 3 + '╦'
    top_line += '═' * 3 + '╗'
    mid_line = '╠'
    for i in range(n - 1):
        mid_line += '═' * 3 + '╬'
    mid_line += '═' * 3 + '╣'    
    bot_line = '╚'
    for i in range(n - 1):
        bot_line += '═' * 3 + '╩'
    bot_line += '═' * 3 + '╝'
    # Print the top of the matrix
    print(top_line)
    # Iterate through the rows and columns of the matrix, and print
    # each element with proper padding
    for row in range(n):
        line = '\u2551'
        for col in range(n):
            element = Solution[f'V{row}{col}']
            line += f' {element} ║'
        print(line)
        # Print a horizontal line
        if row < n - 1:
            print(mid_line)
    # Print the bottom of the matrix
    print(bot_line)

In [25]:
show(Solution)

╔═══╦═══╦═══╦═══╦═══╦═══╗
║ 0 ║ 1 ║ 2 ║ 3 ║ 4 ║ 5 ║
╠═══╬═══╬═══╬═══╬═══╬═══╣
║ 1 ║ 2 ║ 0 ║ 4 ║ 5 ║ 3 ║
╠═══╬═══╬═══╬═══╬═══╬═══╣
║ 2 ║ 0 ║ 1 ║ 5 ║ 3 ║ 4 ║
╠═══╬═══╬═══╬═══╬═══╬═══╣
║ 3 ║ 5 ║ 4 ║ 0 ║ 2 ║ 1 ║
╠═══╬═══╬═══╬═══╬═══╬═══╣
║ 4 ║ 3 ║ 5 ║ 1 ║ 0 ║ 2 ║
╠═══╬═══╬═══╬═══╬═══╬═══╣
║ 5 ║ 4 ║ 3 ║ 2 ║ 1 ║ 0 ║
╚═══╩═══╩═══╩═══╩═══╩═══╝
